In [1]:
import PyPDF2
import re
import os
from tkinter import filedialog
import tkinter as tk
# import pandas as pd
# import openpyxl as op
import xlwings as xw
from tkinter import simpledialog
# import fitz
# from fitz import Document, Page, Rect
import os.path
from selenium import webdriver
from selenium.webdriver import ActionChains
from time import sleep
import random
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait as wait
import xlsxwriter

In [2]:
root = tk.Tk()
root.wm_attributes('-topmost', 1)
root.withdraw()
file_full = tk.filedialog.askopenfilename(title= "Select Standard BOM")
file_type = os.path.splitext(file_full)[1]
list_path = os.path.dirname(file_full)
wb = xw.Book(file_full)
ws = wb.sheets.active
lr = ws.range('C' + str(wb.sheets[0].cells.last_cell.row)).end('up').row
# pn_list = ws.range('B1:B' + str(lr)).value

In [3]:
get_info = tk.messagebox.askquestion('Get info',\
            'do you want to get RoHS and FRU info?',\
            icon='question')
download_attached = tk.messagebox.askquestion('Download Attached',\
            'do you want to Download Attached?',\
            icon='question')

In [4]:
# driver = webdriver.Edge()
# actions = ActionChains(driver)
# #docmentation/report/Unrel Part number revision history
# edoc_url = "http://rdt.kla-tencor.com/rdt2/edoc_ei/scripts/eiPartRevHistory.asp?txtProduct=EDOC&Tab=4"
# driver.get(edoc_url)
# print('opening Edoc ...')
# edoc_page = driver.current_window_handle
# sleep(2)
# driver.switch_to.window(edoc_page)




In [5]:
# mpn

In [6]:
# lr = ws.range('B' + str(wb.sheets[0].cells.last_cell.row)).end('up').row
# # print('lst row:',lr)
# for i in range(2,lr+1):
#     mpn = str(ws.range(i,2).value) #.strip()
#     driver.switch_to.window(edoc_page)
#     search_input = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
#         '/html/body/table/tbody/tr/td/table[2]/tbody/tr/td[2]/form[3]/table/tbody/tr[1]/td[2]/input')))
# #     mpn = 'DF-HSL-1524'
#     search_input.clear()
#     search_input.send_keys(mpn+'*')
#     sleep(1)
#     search_input.send_keys(Keys.RETURN)
#     driver.switch_to.window(driver.window_handles[-1])
    
#     table = driver.find_elements(By.XPATH, \
#             '/html/body/table/tbody/tr/td/table[2]/tbody/tr/td[2]/table[2]/tbody/tr')
#     if len(table) < 1:
#         driver.switch_to.window(driver.window_handles[-1])
#         back_btn = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH,\
#         '/html/body/table/tbody/tr/td/table[2]/tbody/tr/td[2]/form/table/tbody/tr[3]/td/div/input')))
#         back_btn.click()
#         ws.range(i,6).value = 'Not found in EDOC'
#     elif len(table) > 2:
#         ask = tk.messagebox.askokcancel('Confirm','Please check:')
#         print('more than one PNs found')
#     else:
#         driver.switch_to.window(driver.window_handles[-1])
#         print(table[1].text)
#         ws.range(i,6).value = table[1].text
        
        
#         back_btn = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH,\
#             '/html/body/table/tbody/tr/td/table[2]/tbody/tr/td[2]/form/table/tbody/tr/td/div/input')))

#         back_btn.click()
    
    

In [7]:
#check on Enovia
driver = webdriver.Edge()
actions = ActionChains(driver)
enovia_url = 'https://3dx.kla-tencor.com/3dspace/common/KLA_emxNavigator.jsp?ticket=ST-56034-MgBi6B0tOkgnvCEwahkW-cas&collabSpace=KLA+Collabspace'
driver.get(enovia_url)
print('opening Enovia ...')
enovia_page = driver.current_window_handle
if download_attached == "yes":
    ask = tk.messagebox.askokcancel('Setting','Please turn on setting/Cookies and site permissions/PDF documents/Always download PDF files')

opening Enovia ...


In [8]:
# driver.switch_to.window(driver.window_handles[-1])
# enovia_page = driver.current_window_handle

In [35]:
col_PN = 2
row_PN = 2
col_des = 4
col_des = False
for row in range(1,5):
    for col in range(1,7):
        
#         print(type(ws.range(row,col).value))
        if ws.range(row,col).value is not None and type(ws.range(row,col).value) == str:
            print(row,col,ws.range(row,col).value)
            if "pn" in ws.range(row,col).value.strip().lower() \
            or "part number" in ws.range(row,col).value.strip().lower():
                print('PN row-col', row, col)
                row_PN = row
                col_PN = col
            if ws.range(row,col).value.strip() == "MPN":
                print('MPN row-col', row, col)
            if "description" in ws.range(row,col).value.strip().lower():
                col_des = True
                col_des = col

1 1 Assembly # (BOM)
1 3 0657645-000
1 4 ASSY,OUTRIGGER TEST FIXTURE,SP7
1 5 Date
2 1 Level
2 2 Find Number
2 3 Part Number
PN row-col 2 3
2 4 Part Description
2 5 Quantity
2 6 Not found in EDOC
3 3 0657635-000
3 4 WELDED FRAME,FIXTURE,OUTRIGGER,SP7
3 6 Not found in EDOC
4 3 0657654-000
4 4 WELDED,HOLDER  2 GEARBOX, SCAN SIDE
4 6 Not found in EDOC


In [10]:
col_pn_leter = xlsxwriter.utility.xl_col_to_name(col_PN-1)
col_pn_leter

'C'

In [11]:
lr = ws.range(col_pn_leter + str(wb.sheets[0].cells.last_cell.row)).end('up').row
# print('lst row:',lr)
pn_preliminary = []
for i in range(row_PN+1,lr+1):
    pn = str(ws.range(i,col_PN).value) #str(part_find).strip().split()[0].split('_')[0]
    print(i, "PN:", pn)
    if pn == '' or not pn[:7].isnumeric():
        continue
    pn = str(pn).strip().split()[0].split('_')[0]
    driver.switch_to.window(enovia_page)
    search_field = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
            '//*[@id="quickSearch"]')))
    search_field.clear()
    search_field.send_keys(pn[:7]+'*')
#     sleep(1)
    search_field.send_keys(Keys.RETURN)
    sleep(2)
    driver.switch_to.window(driver.window_handles[-1])
    result_win = driver.current_window_handle
    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tableContentFrame'))
#     driver.switch_to.frame('tableContentFrame')
    driver.switch_to.frame('tableBodyRight')
    affected_items = driver.find_elements(By.XPATH, '/html/body/form/div[2]/div[2]/table/tbody/tr')
    print('len(affected_items)',len(affected_items))
    for j in range(len(affected_items),0,-1):
        pn_enovia = WebDriverWait(driver, 2).until\
            (EC.presence_of_element_located((By.XPATH, '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                        + str(j) + ']/td[4]')))
#         /html/body/form/div[2]/div[2]/table/tbody/tr[11]/td[18]/a
#             /html/body/form/div[2]/div[2]/table/tbody/tr[11]/td[16]
        sap_type = WebDriverWait(driver, 20).until\
            (EC.presence_of_element_located((By.XPATH, '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                        + str(j) + ']/td[16]')))
        print(j,'sap_type:',sap_type.text)
        print('PN/Eno_PN:',pn,'/',pn_enovia.text)
        if pn[:-3] == pn_enovia.text[:-3] and sap_type.text != '': #
            print(affected_items[j-1].text)
            des_eno = WebDriverWait(driver, 2).until\
            (EC.presence_of_element_located((By.XPATH, '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                        + str(j) + ']/td[10]/span')))
            status_eno = WebDriverWait(driver, 2).until\
            (EC.presence_of_element_located((By.XPATH, '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                        + str(j) + ']/td[6]/a')))
            
            ws.range(i,20).value = des_eno.text
            ws.range(i,21).value = status_eno.text
            if status_eno.text == 'Preliminary':
                pn_preliminary.append([i,pn])
            # if download selected
            if download_attached == "yes":
                try:
                    attached = WebDriverWait(driver, 2).until\
                    (EC.presence_of_element_located((By.XPATH, '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                                + str(j) + ']/td[11]/a')))
                    actions.move_to_element(attached).perform()
                    attached.click()
                    print('Clicked to attached')
                    sleep(2)
                    driver.switch_to.window(driver.window_handles[-1])
                    attached_win = driver.current_window_handle
                    wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tvcInlineFrame_0'))
                    files = driver.find_elements(By.XPATH,  '/html/body/div[2]/div/div')
                    print('num of files:',len(files))
                    file_type = ""
                    for k in range(1,len(files)):
                        driver.switch_to.window(attached_win)
                        wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tvcInlineFrame_0'))
                        files = driver.find_elements(By.XPATH,  '/html/body/div[2]/div/div')
                        file_type = WebDriverWait(driver, 2).until\
                            (EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div['\
                                        +str(k+1) + ']')))
                        print('file_type:',file_type.text)
                        if file_type != "": #'.pdf' in file_type.text:
                            download = WebDriverWait(driver, 2).until\
                            (EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div['\
                                                +str(k+1) + ']')))
                            actions.move_to_element(download).perform()
                            download.click()
                            download_icon = WebDriverWait(driver, 2).until\
                            (EC.presence_of_element_located((By.XPATH, '/html/body/div[2]/div/div['\
                                                +str(k+1) + ']/div/img[1]')))
                #             actions.move_to_element(download).perform()
                            download_icon.click()
                            sleep(2)
                            driver.switch_to.window(driver.window_handles[-1])
                            if driver.current_window_handle != result_win \
                            and driver.current_window_handle != enovia_page:
                                download_win = driver.current_window_handle
                            print('clicked download')
                            sleep(4)
                            ws.range(i,24).value = 'has downloaded document'
    #                         break
                        try:
                            driver.switch_to.window(download_win)
                            driver.close()
                        except:
                            pass
                    try:
                        driver.switch_to.window(driver.window_handles[-1])
                        if driver.current_window_handle != result_win \
                        and driver.current_window_handle != enovia_page:
                            driver.close()
                    except:
                        pass
                except:
                    print('can not down ',pn)
                    pass
            if get_info == 'yes':
                driver.switch_to.window(result_win)
                print('result_win:',result_win)
                driver.switch_to.frame('tableContentFrame')
                driver.switch_to.frame('tableBodyRight')
    #             /html/body/form/div[2]/div[2]/table/tbody/tr[4]/td[1]/a
                open_part = WebDriverWait(driver, 2).until\
                (EC.presence_of_element_located((By.XPATH, '/html/body/form/div[2]/div[2]/table/tbody/tr['\
                            + str(j) + ']/td[1]/a')))
                print('open_part',open_part)
                actions.move_to_element(open_part).perform()
                open_part.click()
                sleep(1)

                driver.switch_to.window(driver.window_handles[-1])
                if driver.current_window_handle != result_win \
                and driver.current_window_handle != enovia_page:
                    detail_win = driver.current_window_handle
                    print('found detail win',detail_win)
                else:
                    for handle in driver.window_handles:
                        if handle != result_win \
                        and handle != enovia_page:
                            detail_win = handle
                            break
    #                 else:
    #                     print('can not find detail win')
                print('detail_win:',detail_win)
                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
    #             /html/body/div[3]/div/div/div[3]/div[1]/ul/li[1]/div/a[1]
                pn_detail = WebDriverWait(driver, 20).until\
                (EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/div/div/div[3]/div[1]/ul/li[1]/div/a[1]')))
                print('pn_detail',pn_detail)
                pn_detail.click()
                sleep(2)
    #             print('driver.page_source',driver.page_source)  
                driver.switch_to.window(detail_win)
                driver.maximize_window()
                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('content'))
                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('detailsDisplay'))
                wait(driver, 100).until(EC.frame_to_be_available_and_switch_to_it('tvcTabs0contentFrame'))
                FRU_pn = WebDriverWait(driver, 50).until\
                (EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/form/div/div[2]/table/tbody/tr[16]/td[2]')))
                print('FRU_pn',FRU_pn.text)
    #             /html/body/div[3]/form/div/div[2]/table/tbody/tr[17]/td[2]
                RoHS_pn = WebDriverWait(driver, 2).until\
                (EC.presence_of_element_located((By.XPATH, '/html/body/div[3]/form/div/div[2]/table/tbody/tr[17]/td[2]')))
                print('RoHS_pn',RoHS_pn.text)
                sleep(5)
                ws.range(i,22).value = FRU_pn.text
                ws.range(i,23).value = RoHS_pn.text
    #             result_win = driver.current_window_handle
                driver.switch_to.window(driver.window_handles[-1])
                if driver.current_window_handle != enovia_page:
                    driver.close()
            try:
                driver.switch_to.window(detail_win)
                driver.close()
                driver.switch_to.window(result_win)
                driver.close()
            except:
                pass
            break
    try:
        driver.switch_to.window(driver.window_handles[-1])
        if driver.current_window_handle != enovia_page:
            driver.close()

    except:
        pass
#     break
print('done ')

3 PN: 0657635-000
len(affected_items) 1
1 sap_type: HALB
PN/Eno_PN: 0657635-000 / 0657635-000
Mechanical Design 0657635-000 AA Preliminary No No No MECHANICAL DESIGN SCM SFS Eagle URRUTIA, DANNY PAUL HALB -000
4 PN: 0657654-000
len(affected_items) 1
1 sap_type: HALB
PN/Eno_PN: 0657654-000 / 0657654-000
Mechanical Assembly 0657654-000 AA Preliminary No No No MECHANICAL ASSEMBLY SCM SFS Eagle URRUTIA, DANNY PAUL HALB -000
5 PN: 0657653-000
len(affected_items) 1
1 sap_type: HALB
PN/Eno_PN: 0657653-000 / 0657653-000
Mechanical Assembly 0657653-000 AA Preliminary No No No MECHANICAL ASSEMBLY SCM SFS Eagle URRUTIA, DANNY PAUL HALB -000
6 PN: 0657640-000
len(affected_items) 2
2 sap_type: HALB
PN/Eno_PN: 0657640-000 / 0657640-000
Mechanical Design 0657640-000 AA Release Pending No No No BLOCK,BASE,FAC,110,TEST,OUTRIGGER,SP7 2 SCM SFS Eagle Tien, Dong Van 3008 (05 - Unreleased) HALB -000 D43826 SP7: LOAD UNREL PARTS Release Pending No
7 PN: 0657638-000
len(affected_items) 2
2 sap_type: HALB
PN/

In [12]:
pn_preliminary

[[3, '0657635-000'], [4, '0657654-000'], [5, '0657653-000']]

In [21]:
# driver = webdriver.Edge()
# actions = ActionChains(driver)
driver.switch_to.window(enovia_page)
edoc_url = "http://rdt.kla-tencor.com/rdt2/edoc_ei/scripts/eiPartRevHistory.asp?txtProduct=EDOC&Tab=4"
driver.get(edoc_url)
print('opening Edoc ...')
edoc_page = driver.current_window_handle
sleep(2)
driver.switch_to.window(edoc_page)

opening Edoc ...


In [42]:
for pn in pn_preliminary:
    mpn = pn[1][:-1] #str(ws.range(i,3).value) #.strip()
    driver.switch_to.window(edoc_page)
    search_input = WebDriverWait(driver, 100).until(EC.presence_of_element_located((By.XPATH,\
        '/html/body/table/tbody/tr/td/table[2]/tbody/tr/td[2]/form[1]/table/tbody/tr[1]/td[2]/input')))
#    /html/body/table/tbody/tr/td/table[2]/tbody/tr/td[2]/form[1]/table/tbody/tr[1]/td[2]/input
    search_input.clear()
    search_input.send_keys(mpn+'*')
    sleep(1)
    search_input.send_keys(Keys.RETURN)
    driver.switch_to.window(driver.window_handles[-1])
    
    table = driver.find_elements(By.XPATH, \
            '/html/body/table/tbody/tr/td/table[2]/tbody/tr/td[2]/table[2]/tbody/tr')
    if len(table) < 1:
        driver.switch_to.window(driver.window_handles[-1])
        back_btn = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH,\
        '/html/body/table/tbody/tr/td/table[2]/tbody/tr/td[2]/form/table/tbody/tr[3]/td/div/input')))
        #/html/body/table/tbody/tr/td/table[2]/tbody/tr/td[2]/form/table/tbody/tr/td/div/input
        back_btn.click()
        ws.range(i,6).value = 'Not found in EDOC'
    elif len(table) > 2:
        ask = tk.messagebox.askokcancel('Confirm','Please check:')
        print('more than one PNs found')
    else:
        driver.switch_to.window(driver.window_handles[-1])
        print(table[1].text)
        if col_des and (table[1].text.split('\n')[-1].strip() != ws.range(pn[0],col_des).value.strip()):
            print(table[1].text.split('\n')[-1])
            ws.range(pn[0],col_des).color = (255,255,0)
        ws.range(pn[0],25).value = table[1].text.replace("\n", "/")
        
        
        back_btn = WebDriverWait(driver, 2).until(EC.presence_of_element_located((By.XPATH,\
            '/html/body/table/tbody/tr/td/table[2]/tbody/tr/td[2]/form/table/tbody/tr/td/div/input')))
        
        back_btn.click()

0657635-000
AA
WELDED FRAME,FIXTURE,OUTRIGGER,SP7
0657654-000
AA
WELDED,HOLDER 2 GEARBOX, SCAN SIDE
WELDED,HOLDER 2 GEARBOX, SCAN SIDE
0657653-000
AA
WELDED,HOLDER 1 GEARBOX,ILUMINIUM SIDE


In [27]:
driver.current_window_handle

'0FDF9D2FAB19FB383E35025E7A182AE8'

In [29]:
driver.switch_to.window(driver.window_handles[-1])

In [30]:
driver.current_window_handle

'0FDF9D2FAB19FB383E35025E7A182AE8'